In [1]:
# Module Importations
import datetime
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
# Project Module Importations
from data_analytics.src.data import cosmos_client_manager
from data_analytics.src.data import load_data
from data_analytics.src.data import test_unit
from data_analytics.src.features import data_munging

In [3]:
# Load data from local store
df_new = load_data.load_dash_dataframe()

Loaded Dash dataframe ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97647 entries, 0 to 97646
Data columns (total 16 columns):
id             97647 non-null object
docType        97647 non-null object
busType        97647 non-null object
proType        97647 non-null object
timeId         97647 non-null object
quantity       97647 non-null float64
curveType      97647 non-null object
resolution     97647 non-null object
setDate        97647 non-null object
setPeriod      97647 non-null float64
powType        97647 non-null object
actFlag        97647 non-null object
docId          97647 non-null object
docRevNum      97647 non-null object
_ts            97647 non-null int64
setDatetime    97647 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(12)
memory usage: 11.9+ MB
None


In [25]:
# Copy dataframe
df_copy = df_new.copy()
df_copy.sort_values(by = ['setDatetime'], inplace = True)

# Mask dataframe between start and end dates
start_date = datetime.datetime(2020,6,1, 0, 0, 0)
end_date = datetime.datetime(2020,8,1, 0, 0, 0)

df_copy = df_copy[(df_copy['setDatetime'] > start_date)]
df_copy = df_copy[(df_copy['setDatetime'] < end_date)]

# Create dict for new dataframe, containing each parameter of interest

# Iterate over dict keys and populate stats

# Create and return df from dict

# Tests
print(start_date)
print(end_date)
print(df_copy)

2020-06-01 00:00:00
2020-08-01 00:00:00
                                                   id  \
22935                            2020-06-01-P02-Solar   
22945                          2020-06-01-P02-Biomass   
22944             2020-06-01-P02-Hydro Pumped Storage   
22942                 2020-06-01-P02-Fossil Hard coal   
22943  2020-06-01-P02-Hydro Run-of-river and poundage   
...                                               ...   
22920  2020-07-31-P48-Hydro Run-of-river and poundage   
22921             2020-07-31-P48-Hydro Pumped Storage   
22922                          2020-07-31-P48-Biomass   
22923                            2020-07-31-P48-Solar   
22917                       2020-07-31-P48-Fossil Oil   

                          docType           busType   proType  \
22935  Actual generation per type  Solar generation  Realised   
22945  Actual generation per type        Production  Realised   
22944  Actual generation per type        Production  Realised   
22942  Actual g